In [113]:
import numpy as np
import matplotlib.pyplot as plt

In [114]:
# modelling the arrival process
# per second converyor belt
# c1 uses the distribution that tells us how likely the time of the next arrival
# a random value. 

In [115]:
# Exponential Random Number Algorithm
def poisson_est(mean):
    r=np.exp(-mean)
    N=0.0
    s=1.0
    s*=np.random.random()
    while s>r:
        N+=1
        s*=np.random.random()
    else:
        return int(N)

In [116]:
# total number of entities conveyor1 delivers per second for 1000 seconds
c1sample=[poisson_est(1)  for i in range(10)]
np.array(c1sample).mean()
c1sample


[2, 0, 1, 0, 2, 0, 1, 0, 0, 2]

In [117]:
# total entities conveyor belt delivers
totalc1=sum(c1sample)
totalc1

8

In [118]:
def conveyor2(time):
    # number of entities conveyor2 delivers over a given time
    return (int(time/1.5))
conveyor2(1000)

666

In [119]:
def failtime(frate,time):
    # per unit time
    time=time%100
    # cdf of failure 
    c_fail=1 - np.exp(-frate* time)
    if(c_fail>=np.random.random()):
        return time
    else: 
        return 0    

In [120]:
def failnow(frate,time):
    # prob of failure in next second
    p_fail=frate*np.exp(-frate*0)+ frate*np.exp(-frate*1)

    if(p_fail>np.random.random()):
        return time
    else: 
        return 0 

In [121]:
def fail_inverse():
    # this gives the times between events
    return (-1/0.01)*np.log(1-np.random.random())

In [122]:
# f=np.array([failnow(0.01,time) for time in range(10000)])
f=[fail_inverse() for i in range(10000)]
f=np.array(f).astype(int)
f=np.cumsum(f)
f=f[f<=10000]
f

array([  65,  338,  422,  638,  758,  830,  877, 1005, 1066, 1066, 1066,
       1118, 1249, 1387, 1455, 1610, 1825, 1923, 1987, 2062, 2361, 2387,
       2653, 2709, 2896, 3016, 3238, 3300, 3341, 3371, 3427, 3592, 3880,
       4021, 4070, 4119, 4272, 4314, 4341, 4413, 4421, 4447, 4803, 4811,
       4898, 4943, 4995, 5097, 5301, 5390, 5407, 5415, 5468, 5629, 5635,
       5743, 5881, 5896, 6144, 6205, 6220, 6336, 6476, 6552, 6769, 6824,
       6864, 6945, 7035, 7120, 7137, 7147, 7259, 7407, 7505, 7526, 7577,
       7620, 7697, 7736, 7766, 7781, 7822, 8061, 8099, 8154, 8203, 8341,
       8351, 8373, 8382, 8614, 8873, 8981, 9077, 9198, 9354, 9381, 9426,
       9482, 9577, 9687, 9783])

In [123]:
# plt.plot(np.arange(0,10000),f)

In [124]:
# for each second, check the 
def simulate(time):
    C1event=[]
    stationFailures=[]
    completed=0
    downtime=0
    # stacks to count waiting entities at each conveyour belt C
    stackC1=[]
    stackC2=[]
    def stackpop(stackC1,stackC2):
        if stackC2[-1]==1:
            stackC2.pop()
        else:
            stackC2[-1]-=1 
        
        if stackC1[-1]==1:
            stackC1.pop() 
        else:
            stackC1[-1]-=1
            
    def addtoC1(time_index,stackC1,C1event):
            if C1event[time_index]==0:
                return
            if len(stackC1)==0:
                stackC1.append(C1event[time_index])
            else:
                stackC1.append(C1event[time_index]+stackC1[-1])

    # add a entity to the stack every 1.5 seconds
    def addtoC2(time_index,stackC2):
        time_index+=1
        if time_index%1.5==0 or (time_index-0.5)%1.5==0:
            if len(stackC2)==0:
                stackC2.append(1)
            else:
                stackC2.append(stackC2[-1]+1)


# simulate events with respect to random variables
    for t in np.arange(time):
        C1event.append(poisson_est(1))
        stationFailures.append(failnow(0.01,t))

    for time_index in np.arange(time):
        if time_index>time-1:
            return completed
        
        addtoC1(time_index,stackC1,C1event)
        addtoC2(time_index,stackC2)

        # if both parts are ready
        if stackC1 and stackC2 and stackC1[-1]>0 and stackC2[-1]>0:
            # check if the machine is not working
            if stationFailures[time_index]>0:
                for standyby in range(0,5):
                    curr=standyby+time_index+1
                    # conveyors run during downtime
                    addtoC1(curr,stackC1,C1event)
                    addtoC2(curr,stackC2)
                downtime+=5
                # machine fails close to the end of the testing time
                if time_index+5>=time-1:
                    return completed

                # count continues 5 seconds after
                time_index+=5
                stackpop(stackC1,stackC2)
            
                completed+=1
            else: #if the machine is working
                # do this or a while loop - while there are parts
                stackpop(stackC1,stackC2)
                completed+=1
        # print("\npost\nstackc2",stackC2,"\nstackc1",stackC1,'\ntime',time_index,'\n',completed,'\n\n')
    return completed,downtime

In [127]:
simulate(10)

In [126]:
# will station1 do all work available instantly or one per unit time, while loop or one per iteration
# can parts + downtime> total time
# end behavior, produce or no
